In [1]:
import os
import glob
import time
from datetime import timedelta, datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

#setting offilne
plotly.offline.init_notebook_mode(connected=True)
import gezi

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
# !more /home/gezi/tmp/rank/data/video_hour_sgsapp_v2/infos/15/2019122104/scores

In [4]:
# import glob
# import traceback
# import gezi
# from gezi import tqdm
# head = 'mid,docid,dur,pred,abid,ori_lr_score,lr_score,show_time,position,video_time,impression_time,interests,rea,pred_click,pred_dur'
# head2 = 'mid,docid,dur,pred,abid,ori_lr_score,lr_score,show_time,position,video_time,impression_time,interests,rea'
# root = '/home/gezi/tmp/rank/data/video_hour_sgsapp_v2/infos/15/'
# l = []
# for file in tqdm(glob.glob(f'{root}/*/scores')):
#     if not gezi.non_empty(file):
#         continue
#     try:
#         hour = os.path.basename(os.path.dirname(file))
# #         print(hour, file)
#         df = pd.read_csv(file, sep='\t')
#         try:
#             df.columns = head.split(',')
#         except Exception:
#             df.columns = head2.split(',')
#         video_time = df.video_time.mean()
#         click_video_time = df[df.dur>0].video_time.mean()
#         dur_per_click = df[df.dur>0].dur.mean()
#         l.append((hour, video_time, click_video_time, dur_per_click))
# #         print(len(l))
#     except Exception:
# #         print(traceback.format_exc())
#         continue
# df = pd.DataFrame.from_dict(l)
# df.columns = ['hour', 'video_time', 'click_video_time', 'dur_per_click']
# df

In [5]:
# import plotly.express as px
# hours = [datetime.strptime(str(x), '%Y%m%d%H') for x in df.hour.values]
# df.hour = hours
# df = df.sort_values(by=['hour'])
# px.line(df, x='hour', y='video_time')

In [6]:
# # fig = px.line(df, x='hour', y='video_time')
# fig = px.line()
# fig.add_scatter(x=df['hour'], y=df['click_video_time'], mode='lines', name='click_video_time')
# fig.add_scatter(x=df['hour'], y=df['video_time'], mode='lines', name='video_time')
# fig.add_scatter(x=df['hour'], y=df['dur_per_click'], mode='lines', name='video_time')
# py.iplot(fig)

In [7]:
abids = [456, 15]
# metric_files = [
#     '/home/gezi/tmp/rank/log/score_info.15.txt',
#     '/home/gezi/tmp/rank/log/score_info.16.txt',
# ]
metric_files = [
    '/home/gezi/tmp/rank/data/video_hour_sgsapp_v2/infos/15/metrics_online.csv',
#     '/home/gezi/tmp/rank/data/video_hour_sgsapp_v2/infos/15/metrics_offline.csv',
    '/home/gezi/tmp/rank/data/tuwen_hour_sgsapp_v2/infos/15/metrics_online.csv',
#     '/home/gezi/tmp/rank/data/tuwen_hour_sgsapp_v2/infos/15/metrics_offline.csv',
#     '/home/gezi/tmp/rank/data/video_hour_sgsapp_v2/infos/16/metrics_offline.csv',
#     '/home/gezi/tmp/rank/data/tuwen_hour_sgsapp_v2/infos/16/metrics_offline.csv',
]

In [8]:
keys = ['gold/auc', 'group/auc', 'group/click/time_auc', 'auc', 'click/time_auc']

In [28]:
df = pd.concat([pd.read_csv(metric_file, error_bad_lines=False, warn_bad_lines=True) for metric_file in metric_files])
df.hour = df.hour.astype(str)
df = df.groupby(['abtest', 'hour'], as_index=False).last()
df = df.sort_values(by=['hour'])
df = df.drop_duplicates()
# name = 'tuwen'
name = 'video'
df = df[df.mark==name]
# df = df[df.hour > '2019121000']
df.name = name
# df.columns

In [29]:
def compare(df,  metric='auc', names='read_ratio',  offline=False, delta=0., ratio=1., abids=abids, smoothing=0.4):
  if isinstance(names, str):
    names = names.split(',')
  if isinstance(abids, str):
    abids = [int(x) for x in abids.split(',')]
  if offline:
    abids = [x * 100 for x in abids]
    
  df_ = df
  for name in names:
    datas = []
    for abid in abids:
        df = df_[df_.abtest==abid]
        if not len(df):
            continue
        data = go.Scatter(
            x=[datetime.strptime(str(x), '%Y%m%d%H') if len(str(x)) == 10 else datetime.strptime(str(x), '%Y%m%d') for x in df.hour],
            y=gezi.smooth(df[metric].values, smoothing),
            mode='lines+markers',
            line_shape='spline',
            line_smoothing=1.,
            name=str(abid) + ':' + metric
        )

        datas.append(data)

    layout = go.Layout(xaxis=dict(type='date'), title=f'{df_.name}:{metric}', hovermode='x')
    fig = go.Figure(data=datas, layout=layout)
    py.iplot(fig)

In [30]:
compare(df, 'gold/auc', smoothing=0.)

In [31]:
compare(df, 'auc', smoothing=0.)

In [32]:
compare(df, 'group/auc')

In [33]:
compare(df, 'group/click/time_auc')

In [34]:
compare(df, 'click/time_auc')

In [16]:
compare(df, 'click/weighted_time_auc')

In [17]:
compare(df, 'group/click/weighted_time_auc')

In [18]:
compare(df, 'group/click/top3_best')

In [19]:
compare(df, 'group/click/top1_best')

In [20]:
compare(df, 'group/click/top1_rate')

In [21]:
compare(df, 'group/click/top3_best')

In [22]:
compare(df, 'ori/auc')

In [23]:
compare(df, 'cold/auc')

In [24]:
compare(df, 'group/auc')

In [25]:
compare(df, 'stats/time_per_user')

In [26]:
compare(df, 'stats/click_ratio')

In [27]:
compare(df, 'stats/time_per_click', smoothing=0.5)